# WGS sequencing data-processing

***

**Index**

1. QC, trimming, and filtering
   - Quality trimming (Trimmomatic)
   - Re-check quality (FastQC; MultiQC)
   - Host filtering
   - Check library sizes for each step
2. Assembly
   - Co-assembly
   - Per-sample assemblies
   - Assessing assemblies
3. Dereplicating contigs across multiple assemblies
4. Gene prediction of dereplicated contigs
5. Gene annotation of dereplicated contigs
6. Read mapping
   - Generate read mapping index
   - WGS per-sample read mapping
   - WTS per-sample read mapping
7. Assign mapped reads to predicted genes
   - Extract gene coordinates to generate gene_coords SAF file
   - featureCounts: WGS data
   - featureCounts: WTS data
8. Per-sample coverage calculations
   - Coverage calculations overview
   - WGS: per-sample DNA read coverage per gene
   - WTS: per-sample RNA read coverage per gene
9. Final notes
   - Key output files

***

**Data**

DNA folder D1-D6 Waite et al. 2018 samples, D7 = S14, D8 = S15, D9 = S16

For Waite et al. 2018 metagenomes, the three adults were Bonus, Rakiura, and Ellie. For juvenile, Tia, Waa, and Tutoko. The isolates listed on NCBI were cultured from Bonus.

All three DNA samples from OG4727 were cloacitis-affected birds: Merv, Taeatanga and Bravo

## Trimmomatic

**Trimmomatic:prep raw data**

**Pre-processing**: concatenating samples files from multiple lanes
   - concatenate the files for each read (R1 and R2, separately) from multiple lanes into single (paired) files per sample
   - also rename files to a simpler format for downstream use

In [ ]:
# Change to working directory
cd <working_directory>

# Make directory 0.raw_data
mkdir 0.Raw_concat

# Set up variables for input files path and output path
inpath=0.Raw/fastq
outpath=0.Raw_concat

# For each of reads 1 and 2 (R1 and R2), concatenate multiple lanes (L001-L008) files into single output files, and rename based on sampleIDs
for read in R1 R2;
do
    cat ${inpath}/*4727-01-55-1*_${read}_001.fastq.gz > ${outpath}/S1_${read}.fastq.gz
    cat ${inpath}/*4727-02-55-1*_${read}_001.fastq.gz > ${outpath}/S2_${read}.fastq.gz
    cat ${inpath}/*4727-03-55-1*_${read}_001.fastq.gz > ${outpath}/S3_${read}.fastq.gz
    cat ${inpath}/*4727-04-55-1*_${read}_001.fastq.gz > ${outpath}/S4_${read}.fastq.gz
    cat ${inpath}/*4727-05-55-1*_${read}_001.fastq.gz > ${outpath}/S5_${read}.fastq.gz
    cat ${inpath}/*4727-06-55-1*_${read}_001.fastq.gz > ${outpath}/S6_${read}.fastq.gz
    cat ${inpath}/*4727-07-55-1*_${read}_001.fastq.gz > ${outpath}/S7_${read}.fastq.gz
    cat ${inpath}/*4727-08-55-1*_${read}_001.fastq.gz > ${outpath}/S8_${read}.fastq.gz
    cat ${inpath}/*4727-09-55-1*_${read}_001.fastq.gz > ${outpath}/S9_${read}.fastq.gz
    cat ${inpath}/*4727-10-55-1*_${read}_001.fastq.gz > ${outpath}/S10_${read}.fastq.gz
    cat ${inpath}/*4727-11-55-1*_${read}_001.fastq.gz > ${outpath}/S11_${read}.fastq.gz
    cat ${inpath}/*4727-12-55-1*_${read}_001.fastq.gz > ${outpath}/S12_${read}.fastq.gz
    cat ${inpath}/*4727-13-55-1*_${read}_001.fastq.gz > ${outpath}/S13_${read}.fastq.gz
    cat ${inpath}/*4727-14-25-01*_${read}_001.fastq.gz > ${outpath}/S14_${read}.fastq.gz
    cat ${inpath}/*4727-15-25-01*_${read}_001.fastq.gz > ${outpath}/S15_${read}.fastq.gz
    cat ${inpath}/*4727-16-25-01*_${read}_001.fastq.gz > ${outpath}/S16_${read}.fastq.gz
done

# Copy S14, S15 and S16 files to WGS 0.Raw_concat and rename D7, D8 and D9, respectively.

**Trimmomatic: run**

**NOTE**: 
- It's good to also include a search for a truncated version of the adapters, as sometimes these aren't picked up by trimmomatic for fastqc (this is the ILLUMINACLIP bit in the script below)
- Set `CROP` and `MINLENGTH` to something appropriate for your data (relative to sequence length for this sequencing run)

Slurm array for 9 samples

**DNA WGS samples**

`sbatch scripts/wgs_1_trimmomatic.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_1_trimmomatic
#SBATCH --time 00:10:00
#SBATCH --mem=12GB
#SBATCH --array=1-9
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=10
#SBATCH -e slurm_output/wgs_1_trimmomatic_%a_%j.err
#SBATCH -o slurm_output/wgs_1_trimmomatic_%a_%j.out

# Load module(s)
module purge
module load Trimmomatic/0.39-Java-1.8.0_144                

# Change to working directory
cd <working_directory>/WGS/

# Make output directory 
mkdir -p 1.Qual_filtered_trimmomatic/

# Set up variables for input path and output path
inpath=0.Raw_concat
outpath=1.Qual_filtered_trimmomatic

# Make adapter file if not already created
if [ ! -f iua.fna ]; then
    echo ">FastQC_adapter" > iua.fna
    echo "AGATCGGAAGAG" >> iua.fna
fi
           
# Quality filter and trim 
srun trimmomatic PE -threads 10 -phred33 \
${inpath}/D${SLURM_ARRAY_TASK_ID}_R1.fastq.gz ${inpath}/D${SLURM_ARRAY_TASK_ID}_R2.fastq.gz \
${outpath}/D${SLURM_ARRAY_TASK_ID}_R1.fastq D${SLURM_ARRAY_TASK_ID}_R1.single1.fastq \
${outpath}/D${SLURM_ARRAY_TASK_ID}_R2.fastq D${SLURM_ARRAY_TASK_ID}_R2.single2.fastq \
ILLUMINACLIP:iua.fna:1:25:7 CROP:115 SLIDINGWINDOW:4:30 MINLEN:50

# Tidy up the singleton reads
cat D${SLURM_ARRAY_TASK_ID}_R1.single1.fastq D${SLURM_ARRAY_TASK_ID}_R2.single2.fastq \
> ${outpath}/D${SLURM_ARRAY_TASK_ID}_single.fastq

rm D${SLURM_ARRAY_TASK_ID}_R1.single1.fastq D${SLURM_ARRAY_TASK_ID}_R2.single2.fastq

***

## Fastqc analysis of post-trimmed reads

`sbatch scripts/wgs_1_qc_fastqc.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_1_qc_fastqc
#SBATCH --time 01:30:00
#SBATCH --mem 2GB
#SBATCH --array=1-9
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 2
#SBATCH -e slurm_output/wgs_1_qc_fastqc_%a_%j.err
#SBATCH -o slurm_output/wgs_1_qc_fastqc_%a_%j.out

# Set up working directories
cd <working_directory>/WGS/
mkdir -p 1.Qual_filtered_trimmomatic/fastqc/

# load modules
module load FastQC/0.11.9
module load MultiQC/1.9-gimkl-2020a-Python-3.8.2

# Run fastqc on each sample           
srun fastqc \
-o 1.Qual_filtered_trimmomatic/fastqc/ \
1.Qual_filtered_trimmomatic/D${SLURM_ARRAY_TASK_ID}_R1.fastq 1.Qual_filtered_trimmomatic/D${SLURM_ARRAY_TASK_ID}_R2.fastq

**MultiQC**

`sbatch scripts/wgs_1_qc_multiqc.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_1_qc_multiqc
#SBATCH --time 00:10:00
#SBATCH --mem 1GB
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 2
#SBATCH -e slurm_output/wgs_1_qc_multiqc_%a_%j.err
#SBATCH -o slurm_output/wgs_1_qc_multiqc_%a_%j.out

# Set up working directories
cd <working_directory>/WGS/
mkdir -p 1.Qual_filtered_trimmomatic/fastqc/

# load modules
module load FastQC/0.11.9
module load MultiQC/1.9-gimkl-2020a-Python-3.8.2

# Run multiqc to generate report for all samples
srun multiqc -f \
-o 1.Qual_filtered_trimmomatic/fastqc/ \
1.Qual_filtered_trimmomatic/fastqc/

***

## Filter out host sequences

**Preamble**

Metagenome data derived from microbial communities associated with a host should ideally be filtered to remove any reads originating from host DNA. This may improve the quality and efficiency of downstream data processing, and is also an important consideration when working with metagenomes that may include data of a sensitive nature (and which may also need to be removed prior to making the data publicly available e.g. kākāpō). This is especially important for any studies involving human subjects or those involving samples derived from taonga species.

There are several approaches that can be used to achieve this. The general principle is to map your reads to a reference genome (e.g. human genome) and remove those reads that map to the reference from the dataset.


Here, we will map our reads against a masked kākāpō genome which is processed to hide sections that: are presumbed microbial contaminant in the reference; have high homology to microbial genes/genomes (e.g. ribosomes); or those that are of low complexity. This ensures that reads that would normally map to these sections of the kākāpō genome are not removed from the dataset (as genuine microbial reads that we wish to retain might also map to these regions), while all reads mapping to the rest of the genome are removed.

**Create masked kākāpō genome**

Download the reference genome from NCBI: https://www.ncbi.nlm.nih.gov/assembly/GCF_004027225.2/

`sbatch scripts/wgs_1_masked_genome.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_1_maked_genome
#SBATCH --time 02:00:00
#SBATCH --mem 23GB
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 1
#SBATCH -e slurm_output/wgs_1_hostfilt_mapping_index_%j.err
#SBATCH -o slurm_output/wgs_1_hostfilt_mapping_index_%j.out

# Set up working directories
cd <working_directory>

# Load BBMap module
module purge
module load BBMap/38.81-gimkl-2020a

# Build BBMap index of reference genome
srun bbmaask.sh in=ncbi-genomes-2021-12-11/GCF_004027225.2_bStrHab1.2.pri_genomic.fna.gz out=masked_kakapo_genome.fa entropy=0.7

**Host filtering: Build BBMap index**

`sbatch scripts/wgs_1_hostfilt_mapping_index.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_1_hostfilt_mapping_index
#SBATCH --time 00:10:00
#SBATCH --mem 23GB
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 1
#SBATCH -e slurm_output/wgs_1_hostfilt_mapping_index_%j.err
#SBATCH -o slurm_output/wgs_1_hostfilt_mapping_index_%j.out

# Set up working directories
mkdir -p <working_directory>/WGS/1.host_filtered/
cd <working_directory>/WGS/1.host_filtered/

# Load BBMap module
module purge
module load BBMap/38.81-gimkl-2020a

# Build BBMap index of reference genome
srun bbmap.sh ref=<working_directory>/masked_kakapo_genome.fa.gz -Xmx23g

**Host filtering: per-sample BBMap read mapping, slurm array**

Note:
- This step outputs fastq files where reads that map to the reference genome have been filtered out.
- The output from `outu` is the filtered file for downstream use.
- Host filtering here is run as a two step process for each sample: first, on the paired reads (R1 and R2), and then again for the unpaired (single) reads file.
- The parameters are set based on the recomendations for host filtering outlined [here](https://www.seqanswers.com/forum/bioinformatics/bioinformatics-aa/37175-introducing-removehuman-human-contaminant-removal?t=42552)

`sbatch scripts/wgs_1_hostfilt_mapping.sl`

In [ ]:
#!/bin/bash
#SBATCH -A uoa03068
#SBATCH -J wgs_1_hostfilt_mapping
#SBATCH --time 00:45:00
#SBATCH --mem 28GB
#SBATCH --ntasks 1
#SBATCH --array=1-9
#SBATCH --cpus-per-task 32
#SBATCH -e slurm_output/wgs_1_hostfilt_mapping_%a_%j.err
#SBATCH -o slurm_output/wgs_1_hostfilt_mapping_%a_%j.out

# Set up working directories
cd <working_directory>/WGS/

# Load BBMap module
module load BBMap/38.81-gimkl-2020a

## Run bbmap

# Paired reads (R1 and R2)
srun bbmap.sh -Xmx28g -t=32 \
minid=0.95 maxindel=3 bwr=0.16 bw=12 quickmatch fast minhits=2 qtrim=rl trimq=10 untrim \
in1=1.Qual_filtered_trimmomatic/D${SLURM_ARRAY_TASK_ID}_R1.fastq \
in2=1.Qual_filtered_trimmomatic/D${SLURM_ARRAY_TASK_ID}_R2.fastq \
outu1=1.host_filtered/D${SLURM_ARRAY_TASK_ID}_R1_hostFilt.fastq \
outu2=1.host_filtered/D${SLURM_ARRAY_TASK_ID}_R2_hostFilt.fastq

# Unpaired (single) reads
srun bbmap.sh -Xmx28g -t=32 \
minid=0.95 maxindel=3 bwr=0.16 bw=12 quickmatch fast minhits=2 qtrim=rl trimq=10 untrim \
in=1.Qual_filtered_trimmomatic/D${SLURM_ARRAY_TASK_ID}_single.fastq \
outu=1.host_filtered/D${SLURM_ARRAY_TASK_ID}_single_hostFilt.fastq

***

## Checking library sizes

In [ ]:
cd <working_directory>

# All raw files
for file in 0.Raw_concat/*; do
    echo ${file} 
done
for file in 0.Raw_concat/*; do
    zgrep -c '@' ${file} 
done

# Trimmed files
for file in 1.Qual_filtered_trimmomatic/*.fastq; do
    echo ${file}
done
for file in 1.Qual_filtered_trimmomatic/*.fastq; do
    grep -c '@' ${file} 
done

# Host filtered files
for file in 1.host_filtered/*.fastq; do
    echo ${file}
done
for file in 1.host_filtered/*.fastq; do
    grep -c '@' ${file} 
done

***

## Assembly

Assembly via metaSPAdes    

Single- versus co-assemblies

- Depending on your study question and available time and computational resources, you may wish to do single assemblies (i.e. a separate assembly per sample), or some variety of co-assemblies (e.g. full co-assembly (all samples together), or mini co-assemblies (e.g. one assembly of samples from group A and a separate assembly of samples from group B)).
- NOTE:
    - Individual assemblies per sample may result in better assemblies overall
    - Alternatively, co-assemblies may be better at assembling rarer taxa that occur in > 1 sample
    - If following up with read mapping (e.g. mapping WTS reads back to assembled WGS contigs), any more than one single co-assembly at this stage will require a subsequent step to dereplicate assembled contigs across the multiple assemblies. (n.b. You can also combine both individual assemblies and co-assemblies and dereplicate across all assemblies).
- For co-assemblies, input files can be concatenated together via `cat`, e.g:
    - `cat sample1_R1.fastq.gz sample2_R1.fastq.gz sample3_R1.fastq.gz sample4_R1.fastq.gz > for_assembly_A_R1.fastq.gz`

**Run co-assembly**

Concatenate reads for assembly: 

In [ ]:
cd <working_directory>/WGS/

mkdir -p 2.assembly_spades/infiles_concat

cat 1.host_filtered/*_R1_hostFilt.fastq > 2.assembly_spades/infiles_concat/host_filtered_reads_R1.fastq
cat 1.host_filtered/*_R2_hostFilt.fastq > 2.assembly_spades/infiles_concat/host_filtered_reads_R2.fastq
cat 1.host_filtered/*_single_hostFilt.fastq > 2.assembly_spades/infiles_concat/host_filtered_reads_single.fastq

`sbatch scripts/wgs_2.coassembly_spades.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_2.coassembly_spades_HF
#SBATCH --time 48:00:00
#SBATCH --mem=200GB
#SBATCH --ntasks=1
#SBATCH --hint nomultithread
#SBATCH --cpus-per-task=18
#SBATCH -e slurm_output/wgs_2.coassembly_spades_HF_%j.err
#SBATCH -o slurm_output/wgs_2.coassembly_spades_HF_%j.out

# Load module(s)
module purge
module load SPAdes/3.13.1-gimkl-2018b

# Set up working directory
cd <working_directory>/WGS/

# Make output directory 
mkdir -p 2.assembly_spades/coassembly

# Run metaSPAdes
srun spades.py --meta -t 18 -m 200 \
-1 2.assembly_spades/infiles_concat/host_filtered_reads_R1.fastq \
-2 2.assembly_spades/infiles_concat/host_filtered_reads_R2.fastq \
-s 2.assembly_spades/infiles_concat/host_filtered_reads_single.fastq \
-o 2.assembly_spades/coassembly/

**Run individual assemblies as slurm array**

*NB: D1 did not assemble despite multiple attempts*

`sbatch scripts/wgs_2.assembly_spades.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_2.assembly_spades_HF
#SBATCH --time 40:00:00
#SBATCH --mem=150GB
#SBATCH --ntasks=1
#SBATCH --array=1-9
#SBATCH --cpus-per-task=12
#SBATCH -e slurm_output/wgs_2.assembly_spades_HF_%a_%j.err
#SBATCH -o slurm_output/wgs_2.assembly_spades_HF_%a_%j.out

# Load module(s)
module purge
module load SPAdes/3.13.1-gimkl-2018b

# Set up working directory
cd <working_directory>/WGS/

# Make output directory 
mkdir -p 2.assembly_spades/indv_assembly

# Run metaSPAdes
srun spades.py --meta -t 12 -m 150 \
-1 1.host_filtered/D${SLURM_ARRAY_TASK_ID}_R1_hostFilt.fastq \
-2 1.host_filtered/D${SLURM_ARRAY_TASK_ID}_R2_hostFilt.fastq \
-s 1.host_filtered/D${SLURM_ARRAY_TASK_ID}_single_hostFilt.fastq \
-o 2.assembly_spades/indv_assembly/D${SLURM_ARRAY_TASK_ID}_assembly/

***Assembly statistics via BBMap's stats.sh script***


A key thing to take note of from the output of this script is the `contig N/L50` (which are output the wrong way around)

In [ ]:
module purge
module load BBMap/38.73-gimkl-2018b

cd <working_directory>/WGS/

## Run stats.sh

# Individual assemblies
for i in {1..9}; do
     stats.sh in=2.assembly_spades/indv_assembly/D${i}_assembly/scaffolds.fasta
done

# Co-assembly
stats.sh in=2.assembly_spades/coassembly/scaffolds.fasta

***

## Dereplicating contigs across multiple assemblies

If you have more than one single co-assembly, it is necessary to dereplicate similar and/or identical contigs across the multiple assemblies to obtain a single set of dereplicated contigs for downstream steps such as read mapping etc.
Below will cover using BBMap's dedupe.sh.

**Caveat:**

Properly dereplicating contigs across assemblies is a generally tricky process in the case where contigs derived from the same genome overlap but have overhangs off both ends.

e.g. the following two contigs may be from the same genome (or closely related genomes), in which case you would ideally like them to be dereplicated into a single representative contig (esp. if there is a gene or genes in the overlap region that you want to map reads back to; retaining both contigs results in splitting the reads over both contigs, falsely deflating the overall coverage count for that particular gene).

`----------------TACC[...]AGATCAAGGACCAACTGGACC[...]\ [...]ATTGAAGTAGCTACC[...]AGATCAAG------------------`

However, this remains problematic to resolve, and so for now we will accept that the dereplication process might not be ideal.

One upshot is that if you included individual sample assemblies and also a co-assembly across all samples (or multiple mini-coassemblies) you likely increase the odds of having one assembled contig from one of them that covers the full expanse of other contigs from this genome(s) (i.e. ideally the other related contigs are contained within a longer one).

**Concatenate contigs from multiple assemblies**

Note:

- On rare occasions, different assemblies can contain contigs with identical names, which can be problematic for downstream processing.
- Here we will modify contig headers to include the assembly ID (e.g. sampleID or coassemblyID) before concatenating together.

In [ ]:
cd <working_directory>/WGS/

mkdir -p 2.assembly_spades/dedupe/infiles/assemblies/

## Copy assembly files and add binID to contig headers 

# Co-assembly
cp 2.assembly_spades/coassembly/scaffolds.fasta 2.assembly_spades/dedupe/infiles/assemblies/coassembly.fna

# Individual sample assemblies
for i in {2..9}; do
   cp 2.assembly_spades/indv_assembly/D${i}_assembly/scaffolds.fasta 2.assembly_spades/dedupe/infiles/assemblies/D${i}_assembly.fna
done

# Modify contigIDs to include assemblyIDs
for file in 2.assembly_spades/dedupe/infiles/assemblies/*.fna; do
    assemblyID=$(basename ${file} .fna)
    sed -i -e "s/>/>${assemblyID}_/g" ${file}
done

# Concatenate assembled contigs into all_contigs.fna
cat 2.assembly_spades/dedupe/infiles/assemblies/*.fna > 2.assembly_spades/dedupe/infiles/all_contigs.fna

**Dereplicate contigs across all assemblies**

In [ ]:
cd <working_directory>/WGS/

# Load module
module purge
module load BBMap/38.73-gimkl-2018b

# Run dedupe on all assemblies
dedupe.sh threads=8 minidentity=100 overwrite=t \
in=2.assembly_spades/dedupe/infiles/all_contigs.fna \
out=2.assembly_spades/dedupe/dereplicated_contigs.fna

***

## Gene prediction of dereplicated contig set

Predict genes in dereplicated contigs via prodigal

Note: If of interest, you can also predict rRNA genes via metaxa2 (although these tend to be poorly assembled), and/or tRNA genes via aragorn.

`sbatch scripts/wgs_3_gene_calling.sl`

In [ ]:
#!/bin/bash -e
#SBATCH -A uoa03068
#SBATCH -J wgs_3_gene_calling
#SBATCH --time 00:30:00
#SBATCH --mem=1GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=20
#SBATCH -e slurm_output/wgs_3_gene_calling_%j.err
#SBATCH -o slurm_output/wgs_3_gene_calling_%j.out

# Set up working directories
cd <working_directory>/WGS/
mkdir -p 3.gene_calling/

# Load modules etc.
module purge
module load prodigal/2.6.3-GCCcore-7.4.0

# Set up variables
infile='2.assembly_spades/dedupe/dereplicated_contigs.fna'
outfile='3.gene_calling/dereplicated_contigs'

# Prodigal
prodigal -p meta -q -i ${infile} -a ${outfile}.prod.faa -d ${outfile}.prod.fna > /dev/null

# Strip metadata from prodigal output (problematic for some downstream applications)
cut -f1 -d " " ${outfile}.prod.faa > ${outfile}.prod.faa.nometa

***

## Gene annotation of dereplicated contig set

Annotate predicted genes in dereplicated contigs via searches against UniRef, UniProt, Pfam, TIGRfam, and KEGG

Note:

- The databases used below are stored within a Handley group project directory. If you do not have access to this directory, you will need to download and/or build the databases separately.
- The Handley group has a paid licence for the full version of *KEGG* used here. If you are not affiliated with the Handley group, you will need to purchase a licence or use a free alternative.
- The full version of *usearch* similarly requires a paid licence. An alternative is to run the search via *DIAMOND*.


`sbatch scripts/wgs_4_gene_annotation.sl`

In [ ]:
#!/bin/bash
#SBATCH -A uoa03068
#SBATCH -J wgs_4_gene_annotation
#SBATCH --time 05:00:00
#SBATCH --mem 250GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=36
#SBATCH -e slurm_output/wgs_4_gene_annotation_%j.err
#SBATCH -o slurm_output/wgs_4_gene_annotation_%j.out

# Set up directories
cd <working_directory>/WGS/
mkdir -p 4.gene_annotation/

# Load modules and/or software paths
module purge
module load HMMER/3.1b2-gimkl-2017a
export PATH="/nesi/project/uoa02469/Software/usearch:$PATH"
# Database path masks
export KEGG_udb="/nesi/project/uoa02469/Databases/KEGG/kegg.20180523.usearch.udb"
export UNIREF100_udb="/nesi/project/uoa02469/Databases/UniRef100/uniref100.20181026.usearch.udb"
export UNIPROT_udb="/nesi/project/uoa02469/Databases/UniProt/uniprot.20181026.usearch.udb"
export PFAM_hmm="/nesi/project/uoa02469/Databases/PFAM_v32.0/Pfam-A.hmm"
export TIGRFAM_hmm="/nesi/project/uoa02469/Databases/TIGRfam_v14.0/TIGRfam.hmm"

# set up in/out variables
INFILE='3.gene_calling/dereplicated_contigs.prod.faa.nometa'
OUTFILE='4.gene_annotation/dereplicated_contigs'

# Run annotations
srun usearch9.0.2132_i86linux64 -id 0.5 -evalue 0.001 -maxhits 10 -top_hits_only -threads 36 \
                            -db ${KEGG_udb} -userfields query+target+tlo+thi+id+bits+evalue \
                            -usearch_global ${INFILE} \
                            -userout ${OUTFILE}.kegg

srun usearch9.0.2132_i86linux64 -id 0.5 -evalue 0.001 -maxhits 10 -top_hits_only -threads 36 \
                            -db ${UNIREF100_udb} -userfields query+target+tlo+thi+id+bits+evalue \
                            -usearch_global ${INFILE} \
                            -userout ${OUTFILE}.uniref100

srun usearch9.0.2132_i86linux64 -id 0.5 -evalue 0.001 -maxhits 10 -top_hits_only -threads 36 \
                            -db ${UNIPROT_udb} -userfields query+target+tlo+thi+id+bits+evalue \
                            -usearch_global ${INFILE} \
                            -userout ${OUTFILE}.uniprot

srun hmmsearch --tblout ${OUTFILE}.pfam -E 1e-3 --cpu 25 ${PFAM_hmm} ${INFILE} > /dev/null
srun hmmsearch --tblout ${OUTFILE}.tigrfam -E 1e-3 --cpu 25 ${TIGRFAM_hmm} ${INFILE} > /dev/null

**Aggregate annotations**

Run through `annotationAggregator.py` to generate summary annotation table

Note:

- `annotationAggregator.py` developed by David Waite
- Due to licencing requirements (e.g. for *KEGG*), `annotationAggregator.py` is only available for researchers affiliated with the Handley group.

In [ ]:
# Set up directories
cd <working_directory>/WGS/

# Load modules and/or software paths
module purge
source /nesi/project/uoa02469/Software/PipelineSetup/annotation.sh

# Run aggregator - unbinned contigs
annotationAggregator.py --toponly --ident 30 --coverage 70 \
-p 4.gene_annotation/dereplicated_contigs.pfam \
-i 4.gene_annotation/dereplicated_contigs.tigrfam \
-r 4.gene_annotation/dereplicated_contigs.uniref100 \
-u 4.gene_annotation/dereplicated_contigs.uniprot \
-k 4.gene_annotation/dereplicated_contigs.kegg \
2.assembly_spades/dedupe/dereplicated_contigs.fna \
3.gene_calling/dereplicated_contigs.prod.fna \
3.gene_calling/dereplicated_contigs.prod.faa \
4.gene_annotation/dereplicated_contigs.annotation_summary

#---------------------------------------------------------#

# Load software
module purge
export PATH="/nesi/project/uoa02469/Software/annotationaggregator_v0.1:$PATH"
# Working directory
cd <working_directory>/WGS/
### Run aggregator
# aggregate all datasets except lsu
annotationAggregator.py --toponly --ident 30 --coverage 70 \
-p 4.gene_annotation/dereplicated_contigs.pfam \
-i 4.gene_annotation/dereplicated_contigs.tigrfam \
-r 4.gene_annotation/dereplicated_contigs.uniref100 \
-u 4.gene_annotation/dereplicated_contigs.uniprot \
-k 4.gene_annotation/dereplicated_contigs.kegg \
2.assembly_spades/dedupe/dereplicated_contigs.fna \
3.gene_calling/dereplicated_contigs.prod.fna \
3.gene_calling/dereplicated_contigs.prod.faa \
4.gene_annotation/dereplicated_contigs.annotation_summary

***

## Read mapping

**BBMap: Generate read mapping index**

Generate read mapping index based on assembled contigs from wgs data.

*NOTE: if altering the memory allocation, this needs to be modified in both the slurm header and the bbmap.sh script*

`sbatch scripts/wgs_5_read_mapping_index.sl`

In [ ]:
#!/bin/bash
#SBATCH -A uoa03068
#SBATCH -J wgs_5_read_mapping_index
#SBATCH --time 00:02:00
#SBATCH --mem 2GB
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 30
#SBATCH -e slurm_output/wgs_5_read_mapping_index_%j.err
#SBATCH -o slurm_output/wgs_5_read_mapping_index_%j.out

# Set up directories
cd <working_directory>/WGS/
mkdir -p 5.read_mapping
cd 5.read_mapping

# Load dependencies
module purge
module load BBMap/38.90-gimkl-2020a

# Build index - SPAdes assemblies
bbmap.sh -Xmx2g ref=<working_directory>/2.assembly_spades/dedupe/dereplicated_contigs.fna

**BBMap: per-sample WGS read mapping, slurm array**

`sbatch scripts/wgs_6_read_mapping.sl`

In [ ]:
#!/bin/bash
#SBATCH -A uoa03068
#SBATCH -J wgs_6_read_mapping
#SBATCH --time 00:20:00
#SBATCH --mem 70GB
#SBATCH --ntasks 1
#SBATCH --array=1-9
#SBATCH --cpus-per-task 30
#SBATCH -e slurm_output/wgs_6_read_mapping_%a_%j.err
#SBATCH -o slurm_output/wgs_6_read_mapping_%a_%j.out

# Set up working directories
cd <working_directory>/WGS/5.read_mapping/
mkdir -p wgs/

# Load dependencies
module purge
module load BBMap/38.90-gimkl-2020a
module load SAMtools/1.10-GCC-9.2.0

# in/out varibles
WGS_READ_DIR='<working_directory>/WGS/1.host_filtered'
OUTPATH='wgs'

# Run read mapping
srun bbmap.sh \
t=30 -Xmx70g ambiguous=best minid=0.95 \
in1=${WGS_READ_DIR}/D${SLURM_ARRAY_TASK_ID}_R1_hostFilt.fastq \
in2=${WGS_READ_DIR}/D${SLURM_ARRAY_TASK_ID}_R2_hostFilt.fastq \
covstats=${OUTPATH}/D${SLURM_ARRAY_TASK_ID}.wgs.covstats.txt \
statsfile=${OUTPATH}/D${SLURM_ARRAY_TASK_ID}.wgs.statsfile.txt \
out=${OUTPATH}/D${SLURM_ARRAY_TASK_ID}.wgs.sam

# Convert sam to bam
samtools sort -@ 10 -o ${OUTPATH}/D${SLURM_ARRAY_TASK_ID}.wgs.bam ${OUTPATH}/D${SLURM_ARRAY_TASK_ID}.wgs.sam

# Run pileup to extract read counts
pileup.sh \
in=${OUTPATH}/D${SLURM_ARRAY_TASK_ID}.wgs.sam \
rpkm=${OUTPATH}/D${SLURM_ARRAY_TASK_ID}.wgs.covstats_pileup.txt 

**BBMap: per-sample WTS read mapping, slurm array**

Map RNA transcripts (filtered reads; see WTS pipeline) to metagenome contigs

`sbatch scripts/wts_6_read_mapping.sl`

In [ ]:
#!/bin/bash
#SBATCH -A uoa03068
#SBATCH -J wts_6_read_mapping
#SBATCH --time 00:20:00
#SBATCH --mem 20GB
#SBATCH --ntasks 1
#SBATCH --array=1-13
#SBATCH --cpus-per-task 30
#SBATCH -e slurm_output/wts_6_read_mapping%a_%j.err
#SBATCH -o slurm_output/wts_6_read_mapping%a_%j.out

# Set up working directories
cd <working_directory>/WGS/5.read_mapping/
mkdir -p wts/

# Load dependencies
module purge
module load BBMap/38.90-gimkl-2020a
module load SAMtools/1.10-GCC-9.2.0

# in/out varibles
READ_DIR='<working_directory>/WTS/1.host_filtered' #see WTS pipeline for processed metatranscriptomic reads
OUTPATH='wts'

# Run read mapping
srun bbmap.sh \
t=30 -Xmx20g ambiguous=best minid=0.95 \
in1=${READ_DIR}/S${SLURM_ARRAY_TASK_ID}_R1_hostFilt.fastq \
in2=${READ_DIR}/S${SLURM_ARRAY_TASK_ID}_R2_hostFilt.fastq \
covstats=${OUTPATH}/S${SLURM_ARRAY_TASK_ID}.wts.covstats.txt \
statsfile=${OUTPATH}/S${SLURM_ARRAY_TASK_ID}.wts.statsfile.txt \
out=${OUTPATH}/S${SLURM_ARRAY_TASK_ID}.wts.sam

# Convert sam to bam
samtools sort -@ 10 -o ${OUTPATH}/S${SLURM_ARRAY_TASK_ID}.wts.bam ${OUTPATH}/S${SLURM_ARRAY_TASK_ID}.wts.sam

# Run pileup to extract read counts
pileup.sh \
in=${OUTPATH}/S${SLURM_ARRAY_TASK_ID}.wts.sam \
rpkm=${OUTPATH}/S${SLURM_ARRAY_TASK_ID}.wts.covstats_pileup.txt 

***

## Assign mapped reads to predicted genes

Assign mapped DNA and RNA reads to predicted genes via featurecounts

**Generate gene_coords SAF file**

Extract coordinates of prodigal predicted genes and generate gene_coords SAF file (required for downstream use with *featurecounts*)

Note: code here c/o David Waite

In [ ]:
cd <working_directory>/WGS/

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3

import pandas as pd
import re

## Prodigal calls for prok bins
# Import the prodigal file, and extract sequence names/metadata
prodigal_file = '3.gene_calling/dereplicated_contigs.prod.faa'
prodigal_headers = [ line.replace('>', '') for line in open(prodigal_file, 'r') if '>' in line ]

# Define a function for splitting the metadata into row-wise dictionaries
def prodigal_to_dict(file_line):
    gene_name, start, stop, orientation, *rest = file_line.split('#')
    return { 'GeneID': gene_name.strip(), 
             'Chr': re.sub('_\d+$', '', gene_name.strip()),
             'Start': start.strip(),
             'End': stop.strip(),
             'Strand': '+' if int(orientation) == 1 else '-' }

# Convert the prodigal lines into a DataFrame
prodigal_buffer = [ prodigal_to_dict(header) for header in prodigal_headers ]
prok_df = pd.DataFrame(prodigal_buffer)

# Order the columns, and save the file
prok_df = prok_df[ ['GeneID', 'Chr', 'Start', 'End', 'Strand' ] ]
prok_df.to_csv('3.gene_calling/dereplicated_contigs.gene_coords.SAF', sep='\t', index=False)

quit()

**featureCounts: WGS read mapping data**

In [ ]:
# Set up working directories
cd <working_directory>/WGS/
mkdir -p 5.read_mapping/wgs/featurecounts/

# Run featureCounts
/nesi/project/uoa02469/Software/subread-1.6.3-source/bin/featureCounts \
-p -T 6 -t exon -F SAF \
-a 3.gene_calling/dereplicated_contigs.gene_coords.SAF \
-o 5.read_mapping/wgs/featurecounts/wgs_dereplicated_contigs.gene_counts.txt \
5.read_mapping/wgs/*.bam

**featureCounts: WTS read mapping data**

In [ ]:
# Set up working directories
cd <working_directory>/WGS/
mkdir -p 5.read_mapping/wts/featurecounts/

# Run featureCounts
/nesi/project/uoa02469/Software/subread-1.6.3-source/bin/featureCounts \
-p -T 6 -t exon -F SAF \
-a 3.gene_calling/dereplicated_contigs.gene_coords.SAF \
-o 5.read_mapping/wts/featurecounts/wts_dereplicated_contigs.gene_counts.txt \
5.read_mapping/wts/*.bam

***

## Per-sample coverage calculations

Per-sample coverages can be calculated at:

- genome level (e.g. across multiple binned contigs)
- contig level (e.g. for viral genomes represented by a single contig)
- gene level

`summarise_counts.py` (and associated `script summarise_counts.R`) was developed by Michael Hoggard and can output each of these sets of summaries depending on the input data (*pileup* or *featurecounts*) and output options.

However, genome and contig-level summaries are probably not useful here (since we have not binned contigs into putative genomes and/or are not working with viral genomes on a single contig), so we will only generate gene-level coverage stats based on the output from featurecounts.

Below, gene-level coverage summary outputs will be generated for:

- WGS read mapping (indicative of DNA data gene-read-coverages)
- WTS read mapping (indicative of RNA data gene-read-coverages)


*NOTE: if you are including lib.size, you will need to generate two separate mapping files, one for wgs data and one for wts data (as the library size counts will differ)*

- Required columns (spelling of column headers matters)
    - `sampleID`: unique strings that identify each sample (these unique strings must also be present in the counts files file names)
    - `group`: treatment groups for each sample (e.g. 'treatment', 'control'; or 'groupA', 'groupB', 'groupC' etc.)
- Optional column
    - `lib.size`: total library size per sample (total read counts in the quality trimmed and filtered fastq files used for read mapping). (NOTE: this column is required if `--format featurecounts` and `--lib_norm total` (for `--format pileup`, this data is present in the input files output by pileup's `rpkm=...` option))


**WGS: per-sample DNA read coverage, gene level**

`summarise_counts.py` process:

- Extracts DNA read counts per gene from *featurecounts* output
- Calculates coverage normalisation based on several methods (incl. rpkm, tpm, tmm). (Choose your preferred/the most appropriate normalised dataset for your data).
- Outputs:
    - Summary table of per-sample coverage stats per gene
    - Summary table of WGS read counts per sample (Read counts from trimmed filtered reads input into read mapping)
    - Summary table of *EdgeR* analysis of "differentially expressed genes" (DEG) based on pairwise comparisons between sample groups
        - In this case, this will actually be differentially *abundant* genes (rather than expressed genes), since we have input DNA read coverage rather than RNA read coverage here, but as *EdgeR* was designed with transcription data in mind its use here is tentative.

In [ ]:
cd <working_directory>/WGS/

# Load python and R
module purge
module load Python/3.8.2-gimkl-2020a
module load R/3.6.2-gimkl-2020a

# Add location of summarise_counts.py and summarise_counts.R to PATH
export PATH="/nesi/project/uoa02469/custom-scripts/MikeH:$PATH"

# Run summarise_counts
summarise_counts.py \
    --input '5.read_mapping/wgs/featurecounts/wgs_dereplicated_contigs.gene_counts.txt' --format featurecounts \
    --sample_mapping_file 5.read_mapping/wgs/wgs_sample_mapping_file.tsv \ # old vs new data groups
    --lib_norm total \
    --count_threshold 10 \
    --read_counts 5.read_mapping/wgs/wgs_summary_read_counts.tsv \
    --edger_out 5.read_mapping/wgs/wgs_summary_edgeR_glmQLFTest.tsv \
    --output 5.read_mapping/wgs/wgs_summary_count_table.tsv

**WTS: per-sample RNA read coverage, gene level**

The process here:

- Extracts RNA read counts per gene from *featurecounts* output
- Calculates coverage normalisation based on several methods (incl. rpkm, tpm, tmm). (Choose your preferred/the most appropriate normalised dataset for your data).
- Outputs:
    - Summary table of per-sample coverage stats per gene
    - Summary table of WTS read counts per sample (Read counts from trimmed filtered reads input into read mapping)
    - Summary table of *EdgeR* analysis of differentially expressed genes (DEG) based on pairwise comparisons between sample groups

In [ ]:
cd <working_directory>/WGS/

# Load python and R
module purge
module load Python/3.8.2-gimkl-2020a
module load R/3.6.2-gimkl-2020a

# Run summarise_counts
summarise_counts.py \
    --input '5.read_mapping/wts/featurecounts/wts_dereplicated_contigs.gene_counts.txt' --format featurecounts \
    --sample_mapping_file 5.read_mapping/wts/wts_sample_mapping_file.tsv \ # cloacitis groups
    --lib_norm total \
    --count_threshold 10 \
    --read_counts 5.read_mapping/wts/wts_summary_read_counts.tsv \
    --edger_out 5.read_mapping/wts/wts_summary_edgeR_glmQLFTest.tsv \
    --output 5.read_mapping/wts/wts_summary_count_table.tsv

***

## Some key output files for downstream analyses

Each of the below outputs should contain a column of shared unique gene identifiers to enable joining for downstream exploration and analyses (e.g. pairing per-sample coverage and/or differential expression results with predicted annotations results etc.)

- `working/dir/4.gene_annotation/dereplicated_contigs.annotation_summary.aa` Gene annotations summary table (with amino acid sequence)
- `working/dir/5.read_mapping/wgs/wgs_summary_count_table.tsv` WGS per-sample (normalised) gene coverage stats summary table
- `working/dir/5.read_mapping/wgs/wgs_summary_edgeR_glmQLFTest.tsv` WGS EdgeR "differentially ~expressed~ abundant genes" summary table
- `working/dir/5.read_mapping/wts/wts_summary_count_table.tsv WTS` per-sample (normalised) gene coverage stats summary table
- `working/dir/5.read_mapping/wts/wts_summary_edgeR_glmQLFTest.tsv` WTS EdgeR "differentially expressed genes" summary table

***